In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?
import sys

from siuba import *
import pandas as pd
import geopandas as gpd
import gcsfs
import datetime as dt
import time
import shapely

from rt_analysis import rt_parser
from rt_analysis import rt_filter_map_plot

import shared_utils
from calitp_data_analysis.tables import tbls

import pyaml
import yaml

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# date for v2 testing
analysis_date = dt.date(2023, 3, 15)

In [3]:
# tbls.mart_transit_database.dim_provider_gtfs_data() >> head(3)

In [4]:
def build_speedmap_index(analysis_date: dt.date):
    '''
    An index table for tracking down a given org's schedule/rt feeds
    returns LazyTbl
    '''
    analysis_dt = dt.datetime.combine(analysis_date, dt.time(0, 0))
    
    dim_orgs = (tbls.mart_transit_database.dim_organizations()
                >> filter(_._valid_from <= analysis_dt, _._valid_to > analysis_dt)
                >> select(_.source_record_id, _.caltrans_district)
               )
    
    orgs_with_vp = (tbls.mart_transit_database.dim_provider_gtfs_data()
    >> filter(_._valid_from <= analysis_dt, _._valid_to > analysis_dt,
              _.reports_site_assessed, _.vehicle_positions_gtfs_dataset_key != None)
    >> inner_join(_, dim_orgs, on = {'organization_source_record_id': 'source_record_id'})
    >> select(_.organization_itp_id, _.organization_name, _.organization_source_record_id,
             _.caltrans_district, _._is_current)
    >> collect()
    )
    assert not orgs_with_vp.isnull().values.any()
    orgs_with_vp['analysis_date'] = analysis_date
    orgs_with_vp = orgs_with_vp >> distinct(_.organization_name,
                    _.organization_itp_id, _.organization_source_record_id,
                    _.caltrans_district, _._is_current, _.analysis_date)
    return orgs_with_vp

In [5]:
speedmap_index = build_speedmap_index(analysis_date)

In [6]:
# df2 >> filter(_.organization_itp_id.isin([315, 246]))
# # rail operators...

## Check/Run rt_parser

In [7]:
speedmap_index

,organization_name,organization_itp_id,organization_source_record_id,caltrans_district,_is_current,analysis_date
0,City of Ojai,231,reckQmUdXUzHFmlVf,07 - Los Angeles,False,2023-03-15
1,Nevada County,221,reczUcQgqgtMpkpKC,03 - Marysville,True,2023-03-15
2,Sonoma County,314,recDupUxInMUgxeiz,04 - Oakland,False,2023-03-15
3,City of Arcata,18,recaaoqEDvwhcmIVT,01 - Eureka,True,2023-03-15
4,City of Duarte,97,recZm8PD8WIdcDL0M,07 - Los Angeles,True,2023-03-15
...,...,...,...,...,...,...
95,Treasure Island Community Development,485,recnW07VirRBSXgRe,04 - Oakland,True,2023-03-15
96,"University of California, Los Angeles",486,recBW9vkDYWIKPIu4,07 - Los Angeles,False,2023-03-15
97,Capitol Corridor Joint Powers Authority,56,recvEBkSBc7UxlarC,04 - Oakland,True,2023-03-15
98,Emeryville Transportation Management Agency,106,rec75nESVRqpwNiLv,04 - Oakland,True,2023-03-15


In [8]:
def check_intermediate_data(speedmap_index_df):
    
    analysis_date = speedmap_index_df.analysis_date.iloc[0]
    progress_path = f'./_rt_progress_{analysis_date}.parquet'
    already_tried = os.path.exists(progress_path)
    if already_tried:
        print(f'found {progress_path}, resuming')
        speedmap_index_joined = pd.read_parquet(progress_path)
    else:
        ## TODO update util to use v1 or v2 folders on bucket? new util?
        operators_ran = shared_utils.rt_utils.get_operators(analysis_date,
                            speedmap_index_df.organization_itp_id.to_list())
        operators_ran_df = pd.DataFrame.from_dict(
                    operators_ran, orient='index', columns = ['status'])
        operators_ran_df.index.name = 'itp_id'
        speedmap_index_joined = speedmap_index >> inner_join(_,
                                                         operators_ran_df, on={'organization_itp_id': 'itp_id'})
    return speedmap_index_joined

In [39]:
speedmap_index_joined = check_intermediate_data(speedmap_index)

found ./_rt_progress_2023-03-15.parquet, resuming


In [64]:
shared_utils.rt_utils.show_full_df(speedmap_index_joined)

,organization_name,organization_itp_id,organization_source_record_id,caltrans_district,_is_current,analysis_date,status
0,Nevada County,221,reczUcQgqgtMpkpKC,03 - Marysville,True,2023-03-15,already_ran
1,Santa Clara Valley Transportation Authority,294,recC5CT95EufmQCXr,04 - Oakland,True,2023-03-15,failed
2,City of Union City,350,recmB4uxrVLRXYF3L,04 - Oakland,False,2023-03-15,already_ran
3,City of Elk Grove,105,recaJnArpFEk5QooE,03 - Marysville,True,2023-03-15,already_ran
4,Capitol Corridor Joint Powers Authority,56,recvEBkSBc7UxlarC,04 - Oakland,True,2023-03-15,already_ran
5,Mountain View Transportation Management Associ...,217,recIeP8mUucOsbvbz,04 - Oakland,True,2023-03-15,already_ran
6,Orange County Transportation Authority,235,recfma7GNR5lQTTTg,12 - Irvine,False,2023-03-15,already_ran
7,Emeryville Transportation Management Agency,106,rec75nESVRqpwNiLv,04 - Oakland,True,2023-03-15,already_ran
8,Anaheim Transportation Network,14,recsrIZdx5Wt6n3ol,12 - Irvine,True,2023-03-15,already_ran
9,City of Burbank,45,rec6z2ivjTxc8Sag3,07 - Los Angeles,True,2023-03-15,already_ran


In [36]:
def stage_intermediate_data(speedmap_index_joined):
    
    analysis_date = speedmap_index_joined.analysis_date.iloc[0]
    progress_path = f'./_rt_progress_{analysis_date}.parquet'
    _df = speedmap_index_joined.copy()[['organization_itp_id', 'status']]
    _df = _df.set_index('organization_itp_id', drop = True)
    
    for itp_id in speedmap_index_joined.organization_itp_id:
    # clunky loop but ensures df saved to parquet with each run...
        
        itp_id_status = (speedmap_index_joined >> filter(
                            _.organization_itp_id == itp_id)).status.iloc[0]
        if itp_id_status != 'already_ran':
            # try:
            #     # how to pbar in cli?
            #     rt_day = rt_parser.OperatorDayAnalysis(itp_id,
            #                                            analysis_date, pbar)
            #     rt_day.export_views_gcs()
            #     _df.loc[itp_id, 'status'] = 'already_ran'
            # except Exception as e:
            #     print(f'{row.organization_itp_id} failed: {e}')
            #     _df.loc[itp_id, 'status'] = 'failed'

            # temp for testing...
            if itp_id % 100 < 90:
                _df.loc[itp_id, 'status'] = 'already_ran'
            else:
                _df.loc[itp_id, 'status'] = 'failed'
                # break # test partway updates of parquet; works!
            
            speedmap_index_joined = speedmap_index_joined >> select(-_.status)
            # return (_df)
            speedmap_index_joined = speedmap_index_joined >> inner_join(_,
                                        _df, on = 'organization_itp_id')
            speedmap_index_joined.to_parquet(progress_path)
    
    return  

In [51]:
stage_intermediate_data(speedmap_index_joined)

In [52]:
## iterate over itp_ids, attempt running, if failed change value to failed?
## store failures in yml? with error if any??
## proceed with build when all already_ran or failed, again print which failed

In [53]:
speedmap_index_joined = check_intermediate_data(speedmap_index)

found ./_rt_progress_2023-03-15.parquet, resuming


## Prep portfolio yml for site

In [71]:
def stage_rt_portfolio(speedmap_index_joined,
                       input_path = '../portfolio/sites/rt.yml',
                       output_path = '../portfolio/sites/test_rt.yml'):
        
    # make sure intermediate data is ran or at least attempted
    assert speedmap_index_joined.status.isin(['already_ran', 'failed']).all()
    
    with open(input_path) as rt_site:
        rt_site_data = yaml.load(rt_site, yaml.Loader)
    
    chapters_list = []
    speedmap_index_joined = speedmap_index_joined >> arrange(_.caltrans_district)
    for district in speedmap_index_joined.caltrans_district.unique():
        if type(district) == type(None):
            continue
        chapter_dict = {}
        filtered = (speedmap_index_joined
                    >> filter(_.caltrans_district == district,
                             _.status != 'failed')
                    >> arrange(_.organization_name)
                   )
        chapter_dict['caption'] = f'District {district}'
        chapter_dict['params'] = {'district': district}
        chapter_dict['sections'] = \
            [{'itp_id': itp_id} for itp_id in filtered.organization_itp_id.to_list()]
        chapters_list += [chapter_dict]   
        
    parts_list = [{'chapters': chapters_list}]
    rt_site_data['parts'] = parts_list
    
    output = pyaml.dump(rt_site_data)
    with open(output_path, 'w') as rt_site:
        rt_site.write(output)
    
    print(f'portfolio yml staged to {output_path}')
    return

In [72]:
stage_rt_portfolio(speedmap_index_joined)

portfolio yml staged to ../portfolio/sites/test_rt.yml


## run portfolio?

In [ ]:
## os chdir ..
## run script